In [27]:
!pip install requests
import base64

In [28]:
import requests
import datetime
from urllib.parse  import urlencode

In [29]:
client_id = 'f6199291e44c4ab8b55e24ae9973c235'
client_secret = '9777424ef1e54fbea098f1032f8fa7f5'

In [38]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Return a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception('You must set client_id and client_secret')
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        
        client_creds_b64 = self.get_client_credentials()
        return{
            'Authorization':f'Basic {client_creds_b64}'
        }

    def get_token_data(self):
        return{
            'grant_type': 'client_credentials'
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range (200, 299):
            raise Exception ('Could not authenticate client')
            return False
            
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            'Authorization' : f'Bearer {access_token}'
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type = 'albums', version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type = 'albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type = 'artists')
    
    def search(self, query, search_type = 'artist' ):
        headers = self.get_resource_header()
        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q' : query, 'type' : search_type.lower()})
        lookup_url = f'{endpoint}?{data}'
        r = requests.get(lookup_url, headers = headers)
        if r.status_code not in range(200, 299):
            return {}
        
        return r.json()

In [39]:
spotify = SpotifyAPI(client_id, client_secret)

In [45]:
spotify.search('year:2015', search_type = 'track' )

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=year%3A2015&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/77SW9BnxLY8rJ0RciFqkHh'},
       'href': 'https://api.spotify.com/v1/artists/77SW9BnxLY8rJ0RciFqkHh',
       'id': '77SW9BnxLY8rJ0RciFqkHh',
       'name': 'The Neighbourhood',
       'type': 'artist',
       'uri': 'spotify:artist:77SW9BnxLY8rJ0RciFqkHh'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK'

In [41]:
spotify.get_artist('4gzpq5DPGxSnKTe4SA8HAU')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU'},
 'followers': {'href': None, 'total': 30048719},
 'genres': ['permanent wave', 'pop'],
 'href': 'https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU',
 'id': '4gzpq5DPGxSnKTe4SA8HAU',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb3bcd1c4d6f324498d3de6f8d',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051743bcd1c4d6f324498d3de6f8d',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1783bcd1c4d6f324498d3de6f8d',
   'width': 160}],
 'name': 'Coldplay',
 'popularity': 91,
 'type': 'artist',
 'uri': 'spotify:artist:4gzpq5DPGxSnKTe4SA8HAU'}